In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import re, pickle
le = LabelEncoder()
from string import punctuation
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
import xgboost as xgb
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')
#ignore warnings
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt to C:\Users\SRAVANTHI
[nltk_data]     SHOROFF\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\SRAVANTHI
[nltk_data]     SHOROFF\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\SRAVANTHI
[nltk_data]     SHOROFF\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
df = pd.read_excel(r"scom2016-alert-02-06-2020 Updated.xlsx")
df.head()

,Server name,Summary of the alert,Description,MonitoringObjectDisplayName,Priority,Severity,Category,TimeRaised,TimeAdded,Type of the Alert (True or False),Action to be taken,Ignorable or Non-Ignorable
0,Red Hat Enterprise Linux Server release 6.10 (...,ACPI daemon is not running,The ACPI daemon on server ukmlwmsw902.uk.pri.o...,Red Hat Enterprise Linux Server release 6.10 (...,Normal,Error,AvailabilityHealth,2020-05-25 16:30:51,2020-05-25 16:28:53,True,Mail to Wintel Server Applications,Non-Ignorable
1,Data Warehouse Synchronization Service,Alert data collection process unable to write ...,Alert data collection process unable to write ...,Data Warehouse Synchronization Service,Normal,Error,StateCollection,2020-05-29 03:32:10,2020-05-29 02:30:05,True,Raise INC to DB support,Non-Ignorable
2,UKSTHSQL009.uk.pri.o2.com,Alert generation was temporarily suspended due...,A rule has generated 50 alerts in the last 60 ...,UKSTHSQL009.uk.pri.o2.com,Normal,Warning,Alert,2020-05-31 21:17:03,2020-05-31 21:15:06,True,mail to wintel server support,Non-ignorable
3,UKSTHSQL009.uk.pri.o2.com,Alert generation was temporarily suspended due...,A rule has generated 50 alerts in the last 60 ...,UKSTHSQL009.uk.pri.o2.com,Normal,Warning,Alert,2020-05-18 21:36:48,2020-05-18 21:35:36,True,mail to wintel server support,Non-ignorable
4,UKDMZSOM005.uk.pri.o2.com,Alert Parameter Replacement Failure,Failed to replace parameter while creating the...,UKDMZSOM005.uk.pri.o2.com,Low,Warning,Alert,2020-05-24 20:30:49,2020-05-24 20:28:49,True,Can be ignored,Ignorable


In [3]:
df.keys()

Index(['Server name ', 'Summary of the alert', 'Description',
       'MonitoringObjectDisplayName', 'Priority', 'Severity', 'Category',
       'TimeRaised', 'TimeAdded', 'Type of the Alert (True or False)',
       'Action to be taken ', 'Ignorable or Non-Ignorable'],
      dtype='object')

In [4]:
df1 = df[['Summary of the alert','Ignorable or Non-Ignorable']]

In [5]:
df1.keys()

Index(['Summary of the alert', 'Ignorable or Non-Ignorable'], dtype='object')

In [6]:
df1['Ignorable or Non-Ignorable'].value_counts()

Non-ignorable    10615
Ignorable         2539
Non-Ignorable      775
Name: Ignorable or Non-Ignorable, dtype: int64

In [7]:
df1['Ignorable or Non-Ignorable'] = df1['Ignorable or Non-Ignorable'].replace(to_replace='Non-ignorable',value='Non-Ignorable')

In [8]:
df1['Ignorable or Non-Ignorable'].value_counts()

Non-Ignorable    11390
Ignorable         2539
Name: Ignorable or Non-Ignorable, dtype: int64

In [9]:
df1['Ignorable or Non-Ignorable']=df1['Ignorable or Non-Ignorable'].apply(lambda x: 1 if (x)=="Non-Ignorable" else 0)

In [10]:
df1['Ignorable or Non-Ignorable'].value_counts()

1    11390
0     2539
Name: Ignorable or Non-Ignorable, dtype: int64

In [11]:
df1['Summary of the alert'].dropna(how='all')

0                               ACPI daemon is not running
1        Alert data collection process unable to write ...
2        Alert generation was temporarily suspended due...
3        Alert generation was temporarily suspended due...
4                      Alert Parameter Replacement Failure
                               ...                        
13924                         XenApp MP Agent Installation
13925                         XenApp MP Agent Installation
13926                         XenApp MP Agent Installation
13927                         XenApp MP Agent Installation
13928                         XenApp MP Agent Installation
Name: Summary of the alert, Length: 13929, dtype: object

In [12]:
df1['Summary of the alert'].shape

(13929,)

In [13]:
df1['Ignorable or Non-Ignorable'].shape

(13929,)

In [14]:
df1['Ignorable or Non-Ignorable'].dropna(how='all')

0        1
1        1
2        1
3        1
4        0
        ..
13924    0
13925    0
13926    0
13927    0
13928    0
Name: Ignorable or Non-Ignorable, Length: 13929, dtype: int64

In [15]:
df1['Summary of the alert'] = df1['Summary of the alert'].apply(lambda x: x.lower())
# df['Alert'] = df['Alert'].apply(lambda x: x.lower())

In [16]:
df1['Summary of the alert'] = df1['Summary of the alert'].apply(lambda s: re.sub(r"[^a-zA-Z0-9]"," ",s))

In [17]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\SRAVANTHI
[nltk_data]     SHOROFF\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [18]:
lemmatizer=WordNetLemmatizer()
def tokenize(text):
    tokens = nltk.word_tokenize(text)
    tokens =[w for w in tokens if not w in stop_words] # [w for w in
    lemmatized_output = ' '.join([lemmatizer.lemmatize(w) for w in tokens])
    return lemmatized_output

In [19]:
df1['Summary of the alert'].apply(tokenize)

0                                      acpi daemon running
1        alert data collection process unable write dat...
2        alert generation temporarily suspended due man...
3        alert generation temporarily suspended due man...
4                      alert parameter replacement failure
                               ...                        
13924                         xenapp mp agent installation
13925                         xenapp mp agent installation
13926                         xenapp mp agent installation
13927                         xenapp mp agent installation
13928                         xenapp mp agent installation
Name: Summary of the alert, Length: 13929, dtype: object

In [20]:
tfidf_vector=TfidfVectorizer(use_idf=True,max_features=1000)

In [21]:
tf = tfidf_vector.fit_transform(df1['Summary of the alert']).todense()

In [22]:
tf

matrix([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.29065627, 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        ...,
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.51889051],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.51889051],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.51889051]])

In [43]:
pwd

'C:\\Users\\SRAVANTHI SHOROFF\\Desktop\\sravanthi\\SCOM_dump_ML\\scom'

In [23]:
pickle.dump(tf, open("vectorized.pkl", "wb"))

In [24]:
vectorizer = pickle.load(open("vectorized.pkl", "rb"))

In [25]:
#taking X and y
X = vectorizer
y = df1['Ignorable or Non-Ignorable']

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [27]:
glmMod = LogisticRegression(penalty='l1', dual=False, tol=0.0001, C=1.0, fit_intercept=True,
                            intercept_scaling=1, class_weight=None, 
                            random_state=None, solver='liblinear', max_iter=100,
                            multi_class='ovr', verbose=2)
glmMod.fit(X_train, y_train)
y_pred=glmMod.predict(X_test)
print("\n\n***** Logistic Regression classification report:*****\n")
print(classification_report(y_test, y_pred))

[LibLinear]

***** Logistic Regression classification report:*****

              precision    recall  f1-score   support

           0       1.00      0.98      0.99       674
           1       1.00      1.00      1.00      2809

    accuracy                           1.00      3483
   macro avg       1.00      0.99      0.99      3483
weighted avg       1.00      1.00      1.00      3483



In [49]:
filename = 'lrMod1.pkl'
pickle.dump(glmMod, open('lrMod1.pkl', 'wb'))

In [50]:
rfMod = RandomForestClassifier(n_estimators=10, criterion='gini', max_depth=None, min_samples_split=2,
                               min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features='auto',
                               max_leaf_nodes=None, bootstrap=True, oob_score=False, n_jobs=1, 
                               random_state=None, verbose=0)
rfMod.fit(X_train, y_train)
y_pred=rfMod.predict(X_test)
print("\n\n***** RandomForestClassifier report:*****\n")
print(classification_report(y_test, y_pred))



***** RandomForestClassifier report:*****

              precision    recall  f1-score   support

           0       0.99      1.00      0.99       671
           1       1.00      1.00      1.00      2812

    accuracy                           1.00      3483
   macro avg       1.00      1.00      1.00      3483
weighted avg       1.00      1.00      1.00      3483



In [51]:
filename = 'rfMod1.pkl'
pickle.dump(rfMod, open('rfMod1.pkl', 'wb'))

In [52]:
gbMod = GradientBoostingClassifier(loss='deviance', learning_rate=0.1, n_estimators=200, subsample=1.0,
                                   min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, 
                                   max_depth=3,
                                   init=None, random_state=None, max_features=None, verbose=0)
gbMod.fit(X_train, y_train)
y_pred=gbMod.predict(X_test)
print("\n\n***** GradientBoosting report:*****\n")
print(classification_report(y_test, y_pred))



***** GradientBoosting report:*****

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       671
           1       1.00      1.00      1.00      2812

    accuracy                           1.00      3483
   macro avg       1.00      1.00      1.00      3483
weighted avg       1.00      1.00      1.00      3483



In [53]:
filename = 'gbMod1.pkl'
pickle.dump(rfMod, open('gbMod1.pkl', 'wb'))

In [54]:
adaMod = AdaBoostClassifier(base_estimator=None, n_estimators=200, learning_rate=1.0)
adaMod.fit(X_train, y_train)
y_pred = adaMod.predict(X_test)
print("\n\n***** Adaboost report:*****\n")
print(classification_report(y_test, y_pred))



***** Adaboost report:*****

              precision    recall  f1-score   support

           0       0.99      1.00      0.99       671
           1       1.00      1.00      1.00      2812

    accuracy                           1.00      3483
   macro avg       0.99      1.00      1.00      3483
weighted avg       1.00      1.00      1.00      3483



In [55]:
filename = 'adaMod1.pkl'
pickle.dump(rfMod, open('adaMod1.pkl', 'wb'))

In [56]:
from sklearn.ensemble import VotingClassifier
votingMod = VotingClassifier(estimators=[('RandomForrest', rfMod),('LogisticRegression', glmMod),("GradientBoosting",gbMod),("AdaBoost",adaMod)], voting='soft')
votingMod = votingMod.fit(X_train, y_train)
test_labels=votingMod.predict((X_test))
#votingMod.score(X_test_transform, y_test)

print("\n\n***** Model Ensemble score report:*****\n")
print(classification_report(y_test, test_labels))


[LibLinear]

***** Model Ensemble score report:*****

              precision    recall  f1-score   support

           0       1.00      0.99      1.00       671
           1       1.00      1.00      1.00      2812

    accuracy                           1.00      3483
   macro avg       1.00      1.00      1.00      3483
weighted avg       1.00      1.00      1.00      3483



In [57]:
filename = 'votingMod1.pkl'
pickle.dump(rfMod, open('votingMod1.pkl', 'wb'))

In [50]:
test_labels.sum()

633

In [29]:
pwd

'C:\\Users\\SRAVANTHI SHOROFF\\Desktop\\sravanthi\\SCOM_ML_v_3'

In [3]:
10615+775+2539

13929

In [6]:
import re, pickle

In [30]:
pkl_file = 'C:\\Users\\SRAVANTHI SHOROFF\\Desktop\\sravanthi\\SCOM_ML_v_3\\votingMod1.pkl'
with open(pkl_file,'rb') as file:
        votingclassifier = pickle.load(file)

In [36]:
# predict class
y_pred = votingclassifier.predict(X)
print(y_pred)
# print(classification_report(y, y_pred))

[1 1 1 ... 0 0 0]


In [37]:
# print(classification_report(y, y_pred))

In [38]:
# output = pd.DataFrame(columns=['y_pred'],data = y_pred)

In [39]:
# output['y_pred'].value_counts()

In [35]:
y_pred['predicted_value'] = y_pred
   

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [77]:
output['pred_val'] = output['y_pred'].replace(to_replace=1,value='Yes')

In [78]:
# output['pred_val'] = output['y_pred'].replace(to_replace=0,value='No')

In [79]:
output['pred_val'].value_counts()

Yes    11389
0       2540
Name: pred_val, dtype: int64

In [80]:
false_output = output[output['pred_val']==0]

In [84]:
len(false_output)

2540